In [1]:
# !pip install pandas==1.3
# !pip install tqdm==4.61.2
# ! pip install strbalance
import os
import pandas as pd
from tqdm import tqdm, notebook
notebook.tqdm().pandas()
%load_ext autoreload

import twee_utils
import importlib
import tqdm, pandas
importlib.reload(twee_utils)
importlib.reload(tqdm)
importlib.reload(pandas)

0it [00:00, ?it/s]

<module 'pandas' from '/Users/alex/anaconda3/envs/spark/lib/python3.7/site-packages/pandas/__init__.py'>

In [ ]:
# !which pip
# /Users/alex/anaconda3/envs/spark/bin/pip
# %pip install strbalance

## Get Twine Files


In [2]:
downloads = "../game_downloads/"
game_html = "../game_html/"

In [3]:
twee_utils.try_unzip(downloads + 'breengrub.zip', game_html + 'temp')

'../game_html/temp'

In [ ]:
import twee_utils
import importlib
import tqdm, pandas
importlib.reload(twee_utils)
importlib.reload(tqdm)
importlib.reload(pandas)

# Test that we can get a zip, a folder, and an html file this way
html1 = twee_utils.get_html_source(downloads + 'breengrub.zip')
html2 = twee_utils.get_html_source(downloads + 'Iron Kid Interactive Story/')
html3 = twee_utils.get_html_source(downloads + '3_Garde_Final.html')
print(html2)

In [ ]:
!ls ../game_downloads/

In [ ]:
%autoreload 2
import twee_utils
import importlib
importlib.reload(twee_utils)

twine = pd.DataFrame({'file': os.listdir(downloads)})
twine['path'] = twine.file.apply(lambda x: os.path.join(downloads, x))
twine['filetype'] = twine.file.apply(lambda x: x.split('.')[-1] if '.' in x else 'unk')
twine = twine[~twine.filetype.isin(('exe', 'quest', 'apk'))]
twine['html'] = twine.path.progress_apply(lambda x: twee_utils.get_html_source(x))

In [ ]:
twine.to_pickle('twine.pkl')
twine.head(30)

## Load saved twine data

In [4]:
import pandas as pd
twine = pd.read_pickle('twine.pkl')

In [ ]:
valid_twine = twine[twine.html != '']
valid_twine = valid_twine.reset_index()
valid_twine

In [ ]:
valid_twine['is_untweeable'] = valid_twine.html.apply(lambda x: twee_utils.is_untweeable(x))
valid_twine
valid_twine[valid_twine.is_untweeable]
tweeable_twine = valid_twine[valid_twine.is_untweeable]
tweeable_twine

In [ ]:

importlib.reload(twee_utils)
def untwee(s):
    html = s['html']
    file = twee_utils.make_temp_file(html)
    twee, error = twee_utils.untwee(file)
    twee_utils.delete_temp_file(file)
    return twee, error

# head = valid_twine.head(5)

tweeable_twine[['twee', 'twee_error']] = tweeable_twine.progress_apply(untwee, axis=1, result_type='expand')
tweeable_twine
# head['twee'], head['twee_error'] = valid_twine.html.apply(
#     (lambda x: untwee(x)), result_type='expand'
# ).T.values

In [ ]:
tweeable_twine.twee = tweeable_twine.twee.str.decode("utf-8")
tweeable_twine.to_pickle('twee.pkl')

## Read twee pickle

In [ ]:
tweeable_twine = pd.read_pickle('twee.pkl')

## TODO : Fix is_valid_twee
So that we can get process more games

In [ ]:
tweeable_twine['valid_twee'] = tweeable_twine.twee.progress_apply(twee_utils.is_valid_twee)
tweeable_twine.to_pickle('valid_twee.pkl')
tweeable_twine

In [ ]:
tweeable_twine = pd.read_pickle('valid_twee.pkl')

## Look at removing all twee games without macros
This isn't valid, since there aren't enough twee stories

Instead, we create an 'absent_macros' column.
### TODO need to remove other things like stylesheets

In [ ]:
tweeable_twine['macros'] = tweeable_twine.twee.progress_apply(twee_utils.get_macros)
tweeable_twine['absent_macros'] = tweeable_twine.twee.progress_apply(twee_utils.replace_macros)
tweeable_twine

full_with_macros = tweeable_twine[tweeable_twine.macros.apply(len) > 0]
print('with macros', full_with_macros.shape)
full_without_macros = tweeable_twine[tweeable_twine.macros.apply(len) == 0]
print('without macros', full_without_macros.shape)
full_without_macros.head(1)

full_with_macros[['twee', 'macros', 'absent_macros']].sample(10)

# Reset the index, make sure its on the full twee story level now
tweeable_twine = tweeable_twine.reset_index(drop=True, col_level=1)
tweeable_twine = tweeable_twine.reset_index(drop=True)
tweeable_twine

## Get passages the new way and create context

In [ ]:
import analysis
importlib.reload(contextual_tree)
importlib.reload(analysis)
import contextual_tree
from contextual_tree import ContextualTweeTree
importlib.reload(twee_utils)
test = tweeable_twine.head(2)
nodes = test.twee.progress_apply(lambda x: ContextualTweeTree.create(twee=x))

# passages = tweeable_twine.twee.apply(lambda x: [(x, for x in ContextualTweeTree.create(twee=x)]).explode()


In [ ]:
importlib.reload(twee_utils)
passages = test.twee.apply(lambda x: [x for x in twee_utils.split_passages(x)]).explode()

## Get Passages (the old fashioned way)

In [ ]:
# Get passages from the 'absent_macros' column created earlier
importlib.reload(twee_utils)
passages = tweeable_twine.twee.apply(lambda x: [x for x in twee_utils.split_passages(x)]).explode()

In [ ]:
pd.set_option('max_colwidth',200)
print(passages.size)
passages[passages.index==81]
# passages.groupby(passages.index).get_group(1)

In [ ]:
passages.to_pickle('passages.pkl')
passages

# Process and clean passages

In [5]:
passages = pd.read_pickle('passages.pkl')
passages = pd.DataFrame(passages)
passages

,twee
0,"::Debug Silo\n<<set $detkey to ""true"">>\n\n<<s..."
0,::officeupstairs\nHalf of the building has col...
0,"::redsfinal\n<<if $healedrads is ""false"">>\nTh..."
0,::SouthA1\nYou walk for several miles before r...
0,"::opheimletin\n""Name and Purpose! Tell or leav..."
...,...
81,::postscript\nThis twine was written in an hal...
81,::impatient\nInternet is wrecking your attenti...
81,::opinion\nThere are many things to be opinion...
81,::software\nWith each update.\n\nThe support f...


In [6]:
importlib.reload(twee_utils)
max_length = 2048

# Rename twee -> passage
if 'twee' in passages:
    passages['passage'] = passages['twee']
    del passages['twee']
print("Initial shape", passages.shape)

passages['original_passage'] = passages.passage

passages['char_count'] = passages.passage.apply(len)

# Cut out the long passages and useless passages
passages = passages[
    ~passages.passage.apply(lambda x: twee_utils.is_special_passage(x) or twee_utils.is_empty_passage(x))
]
passages.passage = passages.passage.apply(lambda x: x[:max_length])
print("Shape after first clean ", passages.shape)

# Clean the twee - for some reason this takes a long time
# Remove images, duplicate newlines, etc
print("Removing and saving macros")
passages['macros'] = passages.passage.progress_apply(twee_utils.get_macros)
passages['passage'] = passages.passage.progress_apply(twee_utils.replace_obvious_macros)
passages['passage'] = passages.passage.progress_apply(twee_utils.replace_macros)
print("Removing images and such") 
passages['passage'] = passages.passage.progress_apply(twee_utils.clean_twee)
print('Shape after cleaning', passages.shape)

# Split into lines
passages['lines'] = passages.passage.progress_apply(twee_utils.split_lines)
passages['line_count'] = passages.lines.progress_apply(len)
passages['title'] = passages.lines.progress_apply(twee_utils.get_title)
if 'lines' in passages:
    del passages['lines']

# Drop Duplicates
passages.drop_duplicates(subset=['passage'])
print()
print("Final shape", passages.shape)
passages[passages.index==0]


Initial shape (10784, 1)
Shape after first clean  (10332, 3)
Removing and saving macros


/Users/alex/anaconda3/envs/spark/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]

...

  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]


Final shape (10332, 6)


,passage,original_passage,char_count,macros,line_count,title
0,::Debug Silo\n[[Test|silos]],"::Debug Silo\n<<set $detkey to ""true"">>\n\n<<s...",82,"[set $detkey to ""true"", set $poweron to ""true""]",2,::Debug Silo
0,::officeupstairs\nHalf of the building has col...,::officeupstairs\nHalf of the building has col...,233,[textinput $lockernum [[Check locker|officeloc...,4,::officeupstairs
0,::redsfinal\nThe radiation sickness has taken ...,"::redsfinal\n<<if $healedrads is ""false"">>\nTh...",304,"[if $healedrads is ""false"", else, endif]",4,::redsfinal
0,::SouthA1\nYou walk for several miles before r...,::SouthA1\nYou walk for several miles before r...,256,[],4,::SouthA1
0,"::opheimletin\n""Name and Purpose! Tell or leav...","::opheimletin\n""Name and Purpose! Tell or leav...",268,[print $PlayerName],6,::opheimletin
...,...,...,...,...,...,...
0,::strength1\n\\nI hope you're smart because yo...,::strength1\n<<if $pushups < 6 >>\\n<<set $str...,698,"[if $pushups < 6 , set $strength to 1, print $...",10,::strength1
0,"::redkill\n""Hey Kid...You know what you are!?!...","::redkill\n""Hey Kid...You know what you are!?!...",289,[],4,::redkill
0,::wreaked cars\n\\nYou have already searched t...,"::wreaked cars\n\n<<if visited (""wreaked cars""...",524,"[if visited (""wreaked cars"") > 1 , else, set $...",11,::wreaked cars
0,::Speed\n\\n\\n\\nYou have Skill Points Remain...,::Speed\n<<if $strength is 1>>\\n<<set $points...,313,"[if $strength is 1, set $points -= 1, else if ...",6,::Speed


In [303]:
pd.set_option('display.max_rows', 231)
passages[passages.index==1]

,passage,original_passage,char_count,macros,line_count,title
1,::pour out onto the floor.\nYou drop the gun.\nYou dry-retch.\nYou [[turn and leave.|Turn and leave.]],"::pour out onto the floor.\nYou <<replace ""drop the gun."">>drop the gun.\n\nYou <<replace ""dry-retch."">>dry-retch.\n\nYou [[turn and leave.|Turn and leave.]]<<endreplace>><<endreplace>>",180,"[replace ""drop the gun."", replace ""dry-retch."", endreplace, endreplace]",4,::pour out onto the floor.
1,"::killhim\nThe soldiers all around the room jump and recoil, but they do nothing.\nYou just watch the mass of the Breen-grub sag in on itself. Gallons of viscous yellow-white fluid [[pour out onto...","::killhim\n<<set $didkilled = true>><<killbreen>>\n\nThe soldiers all around the room jump and recoil, but they do nothing.\n\nYou just watch the mass of the Breen-grub sag in on itself. Gallons o...",248,"[set $didkilled = true, killbreen]",3,::killhim
1,"::deathlie2\n""What levels?"" Alyx asks, a little too brightly.\nThe Breen-grub extends one of those little robot arms the Combine overlords use instead of hands and gestures vaguely at the equipmen...","::deathlie2\n""What levels?"" Alyx asks, a little too brightly.\n\nThe Breen-grub extends one of those little robot arms the Combine overlords use instead of hands and gestures vaguely at the equipm...",332,[],4,::deathlie2
1,"::or not?\n""I'm not sure there's... anything you can do about that,"" he says. ""Unlesssss... you do it all again.""\nThen the white shifts and changes-- and you're back in the snow, flat on your ass...","::or not?\n""I'm not sure there's... anything you can do about that,"" he says. ""Unlesssss... you do it all again.""\n\nThen the white shifts and changes-- and you're back in the snow, flat on your a...",274,[],4,::or not?
1,::Start\n[[begin]],::Start\n<<begin>>,17,[begin],2,::Start
1,"::deathtruth\n""Breen got fried,"" you say, flatly as you can. And then you launch into an explanation of what you did, how you overloaded the reactor... how the original Breen was engulfed by the i...","::deathtruth\n<<set $deathtruth = true>>""Breen got fried,"" you say, flatly as you can. And then you launch into an explanation of what you did, how you overloaded the reactor... how the original B...",767,"[set $deathtruth = true, revise thatshow ""that's how"", revision thatshow, becomes, endrevision, revision thatshow, becomes, endrevision]",5,::deathtruth
1,::soundoff,::soundoff\n<<set $sound = false>><<soundsettings>>,50,"[set $sound = false, soundsettings]",1,::soundoff
1,"::pleased\n""I'm glad you're... sssssatisfied,"" he manages to say.\nThen the white shifts and changes-- and you're back in the snow, flat on your ass, soaked to the bone.\n[[Alyx is already marchin...","::pleased\n""I'm glad you're... sssssatisfied,"" he manages to say.\n\nThen the white shifts and changes-- and you're back in the snow, flat on your ass, soaked to the bone.\n\n[[Alyx is already mar...",226,[],4,::pleased
1,"::deathtruth2\n""Is that what you wanted to know?"" Alyx demands.\n""No,"" grunts the grub.\n""Look,"" she says. [[""Are you going to let us go?""|breen6]]","::deathtruth2\n""Is that what you wanted to know?"" Alyx demands.\n\n""No,"" grunts the grub.\n\n""Look,"" she says. [[""Are you going to let us go?""|breen6]]",146,[],4,::deathtruth2
1,"::breen1\nAlyx is shaking you.""Hey! //Hey!!""//\nYou're sitting on your ass against the far wall, struggling to breathe. Alyx is giving you that withering look. Behind her you can see the quivering...","::breen1\nAlyx is <<revise shaking ""shaking you."">><<revision shaking>><<becomes>>shaking you.<<endrevision>> ""Hey! //Hey!!""//<<revision shaking>><<becomes>>\n\nYou're sitting on your ass against ...",722,"[revise shaking ""shaking you."", revision shaking, becomes, endrevision, revision shaking, becomes, replace ""directly at it--"", revise slaps ""slaps"", revision slaps, becomes, endrevision, revision ...",5,::breen1


In [188]:
pd.set_option('display.max_rows', 20)

In [11]:
import analysis, contextual_tree
importlib.reload(analysis)
importlib.reload(contextual_tree)
from contextual_tree import ContextualTweeTree
pd.set_option('display.max_rows', 20)
first = list(passages[passages.index == 0].passage)
print(first)
tree, passage_dict = ContextualTweeTree.create(passages=list(first))

['::Debug Silo\n[[Test|silos]]', '::officeupstairs\nHalf of the building has collapsed. To your left is a large locker room, with numbers ranging from 100 to 250\nOpen locker number \n[[Return to Lobby|officearmy]]', '::redsfinal\nThe radiation sickness has taken to much of a toll on your body. You collapse as your organs shutdown. And you were so close. \nYou head up a staircase into Reds base. You reach a small metal room with an intercom.\n[[Use Intercom|redsinter]]', "::SouthA1\nYou walk for several miles before reaching the mountains. You could try and go through them, but it would be at least a weeks trip through them. You'd die before you made it across. \n[[Go North|redexit]]\n[[Go through the mountains|mountainsA1]]", '::opheimletin\n"Name and Purpose! Tell or leave."\n[["My name is I\'m just passing through on my way north"|opheimpass]]\n[["Names LetMeIn and I\'m here to beat up curious rat faced assholes"|opheimasshole]]\n[[Go North|NorthB1]]\n[[Go East|EastA2]]', '::Charact

<class 'list'> []
<class 'list'> ['US', 'USSR', 'United States']
<class 'list'> ['US', 'USSR', 'United States']
in <class 'str'> Fi <class 'str'> Red
<class 'list'> ['US', 'USSR', 'United States']
<class 'list'> ['US', 'USSR', 'United States']
<class 'list'> ['US', 'USSR', 'United States']
<class 'list'> ['US', 'USSR', 'United States']
<class 'list'> ['US', 'USSR', 'United States']
in <class 'str'> Fi <class 'str'> Red
<class 'list'> ['US', 'USSR', 'United States']
in <class 'str'> Fi <class 'str'> Red
<class 'list'> ['US', 'USSR', 'United States']
<class 'list'> ['US', 'USSR', 'United States']
<class 'list'> ['US', 'USSR', 'United States']
<class 'list'> ['US', 'USSR', 'United States']
<class 'list'> ['US', 'USSR', 'United States']
<class 'list'> ['US', 'USSR', 'United States']
<class 'list'> ['US', 'USSR', 'United States']
in <class 'str'> Fi <class 'str'> Dick
<class 'list'> ['US', 'USSR', 'United States']
in <class 'str'> Fi <class 'str'> Dick
<class 'list'> ['US', 'USSR', 'United 

In [12]:
tree.render_root()

::Start
├── :: start [-]
│   └── :: origins [-]
│       ├── :: red [-]
│       │   ├── :: redhelp [-]
│       │   │   └── :: reddo [-]
│       │   │       ├── :: charactercreation [-]
│       │   │       └── :: suspect [-]
│       │   │           └── :: charactercreation [-]
│       │   └── :: fixer [-]
│       │       └── :: redhelp [-]
│       │           └── :: reddo [-]
│       │               ├── :: charactercreation [-]
│       │               └── :: suspect [-]
│       │                   └── :: charactercreation [-]
│       ├── :: redf [-]
│       │   ├── :: redhelp [-]
│       │   │   └── :: reddo [-]
│       │   │       ├── :: charactercreation [-]
│       │   │       └── :: suspect [-]
│       │   │           └── :: charactercreation [-]
│       │   └── :: fixer [-]
│       │       └── :: redhelp [-]
│       │           └── :: reddo [-]
│       │               ├── :: charactercreation [-]
│       │               └── :: suspect [-]
│       │                   └── :: character

In [294]:
from anytree.search import findall
title = list(passages[passages.index == 1].title)[2]
# title = twee_utils.title_to_text(title).lower()
# title = 'suspect'
print(title)
node = findall(tree, lambda node: title in node.name.lower())
print([a.title for a in node])

::redsfinal
[]


In [ ]:
# Check if each passage is valid
passages['valid'] = passages.passage.progress_apply(twee_utils.is_valid_passage)
passages

In [ ]:
# pd.set_option("max_rows", 250)
num_invalid = passages[~passages.valid].shape[0]
print('invalid', passages[~passages.valid].shape)
passages[~passages.valid].sample(50)
assert(num_invalid < 400)
print('valid', passages[passages.valid].shape)

In [ ]:
# Prepare passages for generation
importlib.reload(twee_utils)
valid_passages = passages[passages.valid]
pd.set_option('max_colwidth', 200)
# valid_passages['gen_input'] = valid_passages.passage.apply(twee_utils.twee_to_gen_format)
valid_passages[['prompt', 'completion']] = valid_passages.apply(lambda x: twee_utils.twee_to_gen_format_3(x.passage), axis=1, result_type="expand")
valid_passages

## Export to a file

In [ ]:
# with open('twee_training.txt', 'w') as file:
#     file.writelines(passages.gen_input)

name = 'twee_training_1'
    
if 'prompt' in valid_passages and 'completion' in valid_passages:
    pd.set_option('max_colwidth', 500)
#     gpt_json = valid_passages[['prompt', 'completion']]
    
    with open(f'{name}.json', 'w') as file:
        lines = []
        for i, line in valid_passages.iterrows():
            lines.append(line[['prompt', 'completion']].to_json(orient='columns') + '\n')
        file.writelines(lines)
        
    valid_passages.to_pickle(f'{name}.pkl')
        
print('wrote')

testing = pd.DataFrame(valid_passages.prompt)
testing['title'] = testing.prompt.progress_apply(lambda x: twee_utils.gen_to_twee_format_3(x))
testing['text'] = testing.title.progress_apply(lambda x: twee_utils.title_to_text(x))
testing['text'].to_csv(f'{name}_titles.csv')